# Определение количества ресторанов в доступности у пользователя

Для решения данной задачи я выбрал метод трассировки лучей. 
Данный метод заключается в проведении из точки с координатоми пользователя луча и подсчета пересечений луча с ребром полигона. Если луч пересекает нечетное количество ребер, значит точка находится внутри полигона. Если четное, то снаружи. 

In [ ]:
import pandas as pd

In [ ]:
u_cor = pd.read_csv('user_coordinates.csv')
cor = pd.read_csv('place_zone_coordinates.csv')

### Функция, которая определяет, есть ли у луча точки пересечения с ребром с задаными координатами начала и конца.

In [ ]:
def inter_with_edge(x1_u, x2_u, y_u, x1_s, y1_s, x2_s, y2_s):
    
    xdiff = (x1_u - x2_u, x1_s - x2_s)
    ydiff = (0, y1_s - y2_s)

    def det(x1, y1, x2, y2):
        return x1 * y2 - y1 * x2

    div = det(xdiff[0], ydiff[0], xdiff[1], ydiff[1])
    
    if div == 0:
        return 0
    else:
        d = (det(x1_u, y_u, x2_u, y_u), det(x1_s, y1_s, x2_s, y2_s))
        x = det(d[0], d[1], xdiff[0], xdiff[1]) / div
        y = det(d[0], d[1], ydiff[0], ydiff[1]) / div

        if x >= x1_s and x <= x2_s and y >= y1_s and y <= y2_s and (x > x1_u) == (x2_u > x1_u):
            return True
        else:
            return False

Так как я выбрал горизонталный луч, то могут возникнуть ситуации, где данный луч будет проходить через вершину полигона. Для избежания данной ситуации, я создал функцию, которая задает луч, выходящий из точки под углом 45 градусов к обеим осям. Эта функция будет вызываться тогда, когда "у" координата точки совпадет с "у" координатой любой из вершин.

In [ ]:
def inter_with_edge2(x1_u, x2_u, y_u, x1_s, y1_s, x2_s, y2_s):
    
    xdiff = (x1_u - x2_u, x1_s - x2_s)
    ydiff = (y_u - 2*y_u, y1_s - y2_s)

    def det(x1, y1, x2, y2):
        return x1 * y2 - y1 * x2

    div = det(xdiff[0], ydiff[0], xdiff[1], ydiff[1])
    
    if div == 0:
        return False
    else:
        d = (det(x1_u, y_u, x2_u, 2 * y_u), det(x1_s, y1_s, x2_s, y2_s))
        x = det(d[0], d[1], xdiff[0], xdiff[1]) / div
        y = det(d[0], d[1], ydiff[0], ydiff[1]) / div

        if x >= x1_s and x <= x2_s and y >= y1_s and y <= y2_s and (x > x1_u) == (x2_u > x1_u):
            return True
        else:
            return False

### Функция, определяющиая количество пересечений луча с ребрами полигона.

In [ ]:
def inter_with_poly(x1_u, x2_u, y_u, x, y):
    counter = 0
    for i in range(len(x)):
        if y_u != y[i]:
            if i != (len(x) - 1):
                c = inter_with_edge(x1_u, x2_u, y_u, x[i], y[i], x[i+1], y[i+1])
            else:
                c = inter_with_edge(x1_u, x2_u, y_u, x[i], y[i], x[0], y[0])
            if c == True:
                counter+= 1
        else:
            if i != (len(x) - 1):
                c = inter_with_edge2(x1_u, x2_u, y_u, x[i], y[i], x[i+1], y[i+1])
            else:
                c = inter_with_edge2(x1_u, x2_u, y_u, x[i], y[i], x[0], y[0])
            if c == True:
                counter+= 1
    return counter % 2

In [ ]:
result = 0
for a,row in u_cor.iterrows():
    x1_u = row['loc_lat']
    x2_u = 200
    y_u = row['loc_lon']
    unique = cor['place_id'].unique()
    for i in unique:
        x = []
        y = []
        count = cor[cor['place_id'] == i]['point_number'].count()
        place = cor[cor['place_id'] == i]
        for j in range(len(place)):
            y.append(float(place[place['place_id'] == i][place['point_number'] == j]['loc_lon']))
            x.append(float(place[place['place_id'] == i][place['point_number'] == j]['loc_lat']))
        result = result + inter_with_poly(x1_u, x2_u, y_u, x, y)
    print(a, result)
    result = 0